In [2]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.utils import np_utils
import keras.backend as K
import tensorflow as tf
import time

np.random.seed(7)

# Set Number of Threads used
NUM_THREADS = 8
sess = tf.InteractiveSession(config=tf.ConfigProto(intra_op_parallelism_threads=NUM_THREADS))

#num_CPU = 1
#num_cores = 4
#config = tf.ConfigProto(intra_op_parallelism_threads = num_cores,
#                        inter_op_parallelism_threads = num_cores, 
#                        allow_soft_placement = True,
#                        device_count = {'CPU' : num_CPU})
#session = tf.Session(config=config)

K.set_session(sess)

Using TensorFlow backend.


#### Parameter

In [3]:
batch_size = 1
epochs = 1

#### Prepare Data

In [5]:
# define the raw dataset
alphabet = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
# create mapping of characters to integers (0-25) and the reverse
char_to_int = dict((c, i) for i, c in enumerate(alphabet))
int_to_char = dict((i, c) for i, c in enumerate(alphabet))


# prepare the dataset of input to output pairs encoded as integers
seq_length = 1
dataX = []
dataY = []
for i in range(0, len(alphabet) - seq_length, 1):
    seq_in = alphabet[i:i + seq_length]
    seq_out = alphabet[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
    #print(seq_in, '->', seq_out)
    
# Reshape the NumPy array into format expected by the LSTM networks
# => [samples, time steps, features].
X = np.reshape(dataX, (len(dataX), seq_length, 1))
# normalize to the range 0-to-1
X = X / float(len(alphabet))

# one hot encode the output variable
y = np_utils.to_categorical(dataY)

#### LSTM Model

In [12]:
t0 = time.time()
# Create and fit the model
model = Sequential()
model.add(LSTM(20, batch_input_shape=(batch_size, X.shape[1], X.shape[2]), stateful=True))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
for i in range(300):
    model.fit(X, y, epochs=epochs, batch_size=batch_size, verbose=2, shuffle=False)
    model.reset_states()
    
# Evaluate of the model
scores = model.evaluate(X, y, batch_size=batch_size, verbose=0)
model.reset_states()
print("Model Accuracy: %.2f%%" % (scores[1]*100)) 
print("Training tool {:.0f} sec".format(time.time()-t0))

Epoch 1/1
1s - loss: 3.2727 - acc: 0.0400
Epoch 1/1
0s - loss: 3.2604 - acc: 0.1200
Epoch 1/1
0s - loss: 3.2543 - acc: 0.1200
Epoch 1/1
0s - loss: 3.2483 - acc: 0.1200
Epoch 1/1
0s - loss: 3.2419 - acc: 0.1200
Epoch 1/1
0s - loss: 3.2348 - acc: 0.1200
Epoch 1/1
0s - loss: 3.2264 - acc: 0.1200
Epoch 1/1
0s - loss: 3.2161 - acc: 0.1200
Epoch 1/1
0s - loss: 3.2027 - acc: 0.0800
Epoch 1/1
0s - loss: 3.1847 - acc: 0.0800
Epoch 1/1
0s - loss: 3.1603 - acc: 0.0800
Epoch 1/1
0s - loss: 3.1290 - acc: 0.0800
Epoch 1/1
0s - loss: 3.0943 - acc: 0.0800
Epoch 1/1
0s - loss: 3.0623 - acc: 0.1200
Epoch 1/1
0s - loss: 3.0358 - acc: 0.1200
Epoch 1/1
0s - loss: 3.0157 - acc: 0.0800
Epoch 1/1
0s - loss: 3.0044 - acc: 0.1200
Epoch 1/1
0s - loss: 3.0058 - acc: 0.1200
Epoch 1/1
0s - loss: 3.0186 - acc: 0.1600
Epoch 1/1
0s - loss: 3.0227 - acc: 0.1200
Epoch 1/1
0s - loss: 2.9984 - acc: 0.0800
Epoch 1/1
0s - loss: 2.9637 - acc: 0.0800
Epoch 1/1
0s - loss: 2.9314 - acc: 0.0800
Epoch 1/1
0s - loss: 2.8923 - acc:

In [9]:
# demonstrate some model predictions
seed = [char_to_int[alphabet[0]]]
for i in range(0, len(alphabet)-1):
    x = np.reshape(seed, (1, len(seed), 1))
    x = x / float(len(alphabet))
    prediction = model.predict(x, verbose=0)
    index = np.argmax(prediction)
    print(int_to_char[seed[0]], "->", int_to_char[index])
    seed = [index]
model.reset_states()

A -> C
C -> D
D -> E
E -> F
F -> F
F -> H
H -> H
H -> J
J -> K
K -> L
L -> M
M -> N
N -> O
O -> P
P -> P
P -> R
R -> R
R -> T
T -> U
U -> V
V -> W
W -> X
X -> Y
Y -> Z
Z -> Z


In [11]:
# demonstrate a random starting point
letter = "Y"
seed = [char_to_int[letter]]
print("New start: ", letter)
for i in range(0, 5):
    x = np.reshape(seed, (1, len(seed), 1))
    x = x / float(len(alphabet))
    prediction = model.predict(x, verbose=0)
    index = np.argmax(prediction)
    print(int_to_char[seed[0]], "->", int_to_char[index])
    seed = [index]
model.reset_states()

New start:  Y
Y -> B
B -> C
C -> D
D -> E
E -> E
